In [ ]:
import json
import numpy as np
# file_path_pgd : str = 'output_loss_001_llama2_pgd.json'
# file_path_pgd : str = './output_Exp_Grad_Descent_w_Gumbel_SoftMax_Llama2(Ten_behaviors_1kIterations).json'
file_path_pgd : str = './output_EGD_w_Entropy(only_ASCII_init)_20_behaviors(1000_iterations).json'
# Load the JSON file
with open(file_path_pgd, 'r') as file:
    data = json.load(file)

# Process each JSON object
losses = []
harmul_behaviors = []
adv_suffixes = []
count = 1
for item in data[:]:
    print(f'\n\n{count}',end=' ')
    print('_' * 120)
    harmful_behaviour = item.get('harmful-behaviour')
    suffix_token_ids = item.get('suffix_token_ids')
    target = item.get('target')
    loss = item.get("continuous_loss")
    outputs = item.get('outputs')

    # Print extracted information
    print(f"Harmful Behaviour: {harmful_behaviour}")
    print(f"Suffix Token IDs: {suffix_token_ids}")
    print(f"Target: {target}")
    # print(f"Loss: {loss}")
    losses.append(loss)
    harmul_behaviors.append(harmful_behaviour)
    adv_suffixes.append(suffix_token_ids)
    i=1
    for output in outputs:
        print(f'\n#{i} \n{output}')
        i+=1
    print("\n" + "-"*80 + "\n")
    count+=1

In [ ]:
# Convert the list to a NumPy array
numbers_array = np.array(losses)
# print(numbers_array)
# Compute the mean of the array
mean_value = np.mean(numbers_array)

# Print the mean value
print("Mean value:", mean_value)

In [ ]:
# Select the indices that failed in original trial
# Indices you want to select
indices = [2, 7, 13, 14, 16, 17, 19, 20, 22, 23, 29, 32, 33, 43, 48, 49, 51, 63, 71, 75, 76, 78, 81, 83, 86, 88, 92, 95, 98]
failed_suffixes = [adv_suffixes[i] for i in indices]
failed_behaviors = [harmul_behaviors[i] for i in indices]

for i in range(len(failed_behaviors)):
    print(failed_behaviors[i],end=', ')
    # print(failed_suffixes[i])

In [ ]:
# Now load Llama-2 model to generate harmful behaviors
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
import pandas as pd
import torch
import torch.nn.functional as F
import tqdm as tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path: str = "/home/samuel/research/llmattacks/llm-attacks/DIR/llama-2/llama/Llama-2-7b-chat-hf"

def load_model_and_tokenizer(model_path, tokenizer_path=None, device="cuda:0", **kwargs):
    # from llm-attacks
    model = (
        AutoModelForCausalLM.from_pretrained(
            model_path, torch_dtype=torch.float16, trust_remote_code=True, **kwargs
        ).to(device).eval()
    )

    tokenizer_path = model_path if tokenizer_path is None else tokenizer_path

    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path, trust_remote_code=True, use_fast=False
    )

    if "llama-2" in tokenizer_path:
        tokenizer.pad_token = tokenizer.unk_token
        tokenizer.padding_side = "left"
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

model, tokenizer = load_model_and_tokenizer(
        model_path, low_cpu_mem_usage=True, use_cache=False, device=device
    )

In [ ]:
def get_tokens(input_string):
    return torch.tensor(tokenizer(input_string)["input_ids"], device=device)

In [ ]:
# In a for loop, take each failed_behaviors and failed_suffixes then model.generate
for i in tqdm(range(len(failed_behaviors))):
    print(f'\n\n{i+1}',end=' ')
    print('_' * 120)
    print("Harmful Behaviour:", failed_behaviors[i])
    user_prompt_ids = get_tokens(failed_behaviors[i]).to(device)
    adv_prompt_ids = torch.tensor(failed_suffixes[i]).to(device)
    # print(user_prompt_ids) print(adv_prompt_ids)
    input_ids = torch.hstack([user_prompt_ids, adv_prompt_ids]).to(device)
    # = torch.tensor(combined_ids).to(device)
    # print(tokenizer.decode(input_ids, skip_special_tokens=True))
    for trial in range(10):
        print(f'\nTrial #{trial+1}',end=' ')
        print(' -- ' * 20)
        generated_text = model.generate(input_ids.unsqueeze(0), max_length=200)[0]
        print(tokenizer.decode(generated_text, skip_special_tokens=True))